<a href="https://colab.research.google.com/github/rootUJ99/hf_transformer_diffusers/blob/main/transformers.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [22]:
!pip install transformers
!pip install safetensors torch huggingface_hub

In [3]:
from transformers import pipeline
import transformers
print(transformers.__version__)

4.31.0


pipeline -> dont know what is does

In [17]:
generator = pipeline(task="automatic-speech-recognition", model="openai/whisper-small", batch_size=2)

In [15]:
!ls

sample_data


In [18]:
generator("./sample_data/jfk.wav")

/usr/local/lib/python3.10/dist-packages/transformers/generation/utils.py:1369: UserWarning: Using `max_length`'s default (448) to control the generation length. This behaviour is deprecated and will be removed from the config in v5 of Transformers -- we recommend using `max_new_tokens` to control the maximum length of the generation.
  warnings.warn(


{'text': ' And so my fellow Americans, ask not what your country can do for you, ask what you can do for your country.'}

In [8]:
generator(
    [
        "https://huggingface.co/datasets/Narsil/asr_dummy/resolve/main/mlk.flac",
        "https://huggingface.co/datasets/Narsil/asr_dummy/resolve/main/1.flac",
    ]
)

/usr/local/lib/python3.10/dist-packages/transformers/generation/utils.py:1369: UserWarning: Using `max_length`'s default (448) to control the generation length. This behaviour is deprecated and will be removed from the config in v5 of Transformers -- we recommend using `max_new_tokens` to control the maximum length of the generation.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/transformers/generation/utils.py:1369: UserWarning: Using `max_length`'s default (448) to control the generation length. This behaviour is deprecated and will be removed from the config in v5 of Transformers -- we recommend using `max_new_tokens` to control the maximum length of the generation.
  warnings.warn(


[{'text': ' I have a dream that one day this nation will rise up and live out the true meaning of its creed.'},
 {'text': ' He hoped there would be stew for dinner, turnips and carrots and bruised potatoes and fat mutton pieces to be ladled out in thick, peppered, flour-fattened sauce.'}]

### Trying safe tensor (can skip this block for if just looking for )

In [26]:
import safetensors
import os
import datetime
from huggingface_hub import hf_hub_download
from safetensors.torch import load_file
import torch

sf_model = hf_hub_download("gpt2", filename="model.safetensors")
pt_model = hf_hub_download("gpt2", filename="pytorch_model.bin")

start_sf = datetime.datetime.now()
load_file(sf_model)
loadtime_sf = datetime.datetime.now() - start_sf
print("time taken by sf", loadtime_sf)


start_pt = datetime.datetime.now()
torch.load(pt_model)
loadtime_pt = datetime.datetime.now() - start_pt
print("time taken by pt", loadtime_pt)

print("final time taken", loadtime_pt/loadtime_sf)

time taken by sf 0:00:00.013218
time taken by pt 0:00:00.655512
final time taken 49.592374035406266


### Trying classification example

In [32]:
classifier = pipeline("sentiment-analysis")
classifier("here we go again for weath of the society")
classifier([
    "here is the cook, who cooks very good food",
    "he just fucks everything up"
])

No model was supplied, defaulted to distilbert-base-uncased-finetuned-sst-2-english and revision af0f99b (https://huggingface.co/distilbert-base-uncased-finetuned-sst-2-english).
Using a pipeline without specifying a model name and revision in production is not recommended.


[{'label': 'POSITIVE', 'score': 0.9998283386230469},
 {'label': 'NEGATIVE', 'score': 0.9987642765045166}]

### tokanization with autotokenizer

In [36]:
from transformers import AutoTokenizer

model_name = "nlptown/bert-base-multilingual-uncased-sentiment"

tokenizer = AutoTokenizer.from_pretrained(model_name)

encodings = tokenizer("hello this is ujwal the coder")
print(encodings)

{'input_ids': [101, 29155, 10372, 10127, 16889, 29068, 10103, 13121, 10131, 102], 'token_type_ids': [0, 0, 0, 0, 0, 0, 0, 0, 0, 0], 'attention_mask': [1, 1, 1, 1, 1, 1, 1, 1, 1, 1]}
